In [2]:
import numpy as np
import sys
sys.path.append('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/')
from trained_untrained_results_funcs import find_best_layer, loop_through_datasets, load_mean_sem_perf
from plotting_functions import plot_across_subjects
from matplotlib import pyplot as plt
import pandas as pdimport 
import sys
sys.path.append('/home2/ebrahim/beyond-brainscore/')
from banded_reg_func import himalaya_regression_caller

In [3]:
num_seeds = 5

In [4]:
exp = ['243', '384']

br_labels_dict = {}
num_vox_dict = {}
subjects_dict = {}

data_processed_folder_pereira = f'/data/LLMs/data_processed/pereira/dataset'
data_processed_folder_fed = f'/data/LLMs/data_processed/fedorenko/dataset'
data_processed_folder_blank = f'/data/LLMs/data_processed/blank/dataset'

subjects_storage = {}
network_labels_storage = {}

for e in exp:

    bre = np.load(f'{data_processed_folder_pereira}/networks_{e}.npy', allow_pickle=True)
    network_labels_storage[f"pereira{e}"]  = bre
    num_vox_dict[e] = bre.shape[0]
    subjects_storage[f"pereira{e}"] = np.load(f"{data_processed_folder_pereira}/subjects_{e}.npy", allow_pickle=True)

subjects_storage['fedorenko']  = np.load(f"{data_processed_folder_fed}/subjects.npy", allow_pickle=True)
subjects_storage['blank'] = np.load(f"{data_processed_folder_blank}/subjects.npy", allow_pickle=True)

network_labels_storage['fedorenko'] = None
network_labels_storage['blank'] = None

subjects_arr_pereira = np.load(f"{data_processed_folder_pereira}/subjects_complete.npy", allow_pickle=True)
networks_arr_pereira = np.load(f"{data_processed_folder_pereira}/network_complete.npy", allow_pickle=True)

resultsPath_base = '/data/LLMs/brainscore/'
                    

#### Find the best pos value for blank simple model.

#### The code block below does two things. 1st, it resaves the best layer of the untrained model so that I can run it again,
#### because I deleted the yhat values (due to storage space), and I need them now. Second, it stacks the best layer with the simple
#### feature spaces and saves that also.

In [5]:
SL = np.load('/data/LLMs/data_processed/pereira/acts/X_word-num.npz')['layer1']

WP = np.load('/data/LLMs/data_processed/fedorenko/acts/X_pos.npz')['layer_4.7']

WN_POS = np.load('/data/LLMs/data_processed/blank/acts/X_pos-WN.npz')[f'layer_11'] 
WN = np.expand_dims(WN_POS[:, 0],axis=-1)
POS = WN_POS[:, 1:]
np.savez('/data/LLMs/data_processed/blank/acts/X_POS.npz', **{'layer1': POS})
np.savez('/data/LLMs/data_processed/blank/acts/X_WN.npz', **{'layer1': WN})


In [7]:
for d, fe, exp, subjects, networks in loop_through_datasets(dataset_arr=['pereira', 'fedorenko', 'blank'], 
                            feature_extraction_arr=['', '-mp', '-sp']):
    
    print(d, fe, exp)
    
    if networks is not None:
        
        lang_indices = np.argwhere(networks=='language').squeeze()
    
    else:
        
        lang_indices = None
        
    save_acts = {}
    save_acts_stacked = {}

    for i in range(num_seeds):
        
        # load activations
        X_untrained = np.load(f'/data/LLMs/data_processed/{d}/acts/X_gpt2-xl-untrained{fe}_m{i}.npz')
        gpt2_untrained_dict, gpt2_untrained_bl, gpt2_untrained_bl_perf  = find_best_layer(np.arange(49), noL2_str='', exp=exp, 
                                                resultsPath=f"{resultsPath_base}results_{d}/untrained", niter=1,
                                                perf='out_of_sample_r2', feature_extraction=fe, selected_network_indices=lang_indices, 
                                                subjects=subjects, dataset=d, model_name=f'gpt2-xl-untrained', seed_number=i)
        
        print(gpt2_untrained_bl)
        X_best_layer = X_untrained[f'layer_{gpt2_untrained_bl}']
        
        if d == 'pereira':
            
            best_pos = {'384': 3.4, '243': 2.9}
            exp_no_underscore = exp.strip('_')
            position = np.load("/data/LLMs/data_processed/pereira/acts/X_position.npz")[f'layer_{best_pos[exp_no_underscore]}']
            SP_SL = np.hstack((position, SL))
            
            save_acts_stacked[f"m{i}_SP_SL_layer_{gpt2_untrained_bl}"] = np.hstack((X_best_layer, SP_SL))
            save_acts_stacked[f"m{i}_SL_layer_{gpt2_untrained_bl}"] = np.hstack((X_best_layer, SL))
            save_acts_stacked[f"m{i}_SP_layer_{gpt2_untrained_bl}"] = np.hstack((X_best_layer, position))
            
        elif d == 'fedorenko':
            save_acts_stacked[f"m{i}_WP_layer_{gpt2_untrained_bl}"]  = np.hstack((X_best_layer, WP))
        else:
            save_acts_stacked[f"m{i}_POS_WN_layer_{gpt2_untrained_bl}"]  = np.hstack((X_best_layer, WN_POS))
            save_acts_stacked[f"m{i}_POS_layer_{gpt2_untrained_bl}"]  = np.hstack((X_best_layer, POS))
            save_acts_stacked[f"m{i}_WN_layer_{gpt2_untrained_bl}"]  = np.hstack((X_best_layer, WN))
            
        save_acts[f"m{i}_layer_{gpt2_untrained_bl}"] = X_best_layer
        

    np.savez(f"/data/LLMs/data_processed/{d}/acts/X_gpt2-xl-untrained{fe}-r2-best{exp}", **save_acts)
    np.savez(f"/data/LLMs/data_processed/{d}/acts/X_gpt2-xl-untrained{fe}-var-par{exp}", **save_acts_stacked)
    

pereira  _243
1
1
2
1
1
pereira  _384
14
3
3
1
3
pereira -mp _243
0
0
0
0
0
pereira -mp _384
0
0
0
0
0
pereira -sp _243
1
1
1
1
1
pereira -sp _384
0
0
0
0
0
fedorenko  
7
19
0
0
1
fedorenko -mp 
3
0
0
0
0
fedorenko -sp 
3
19
0
0
0
blank  
48
10
17
4
48
blank -mp 
8
3
5
2
3
blank -sp 
1
3
5
2
3
